In [1]:
!git clone https://github.com/franciscogarate/utils_ucm

Cloning into 'utils_ucm'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (230/230), done.
remote: Compressing objects: 100% (184/184), done.
remote: Total 230 (delta 120), reused 118 (delta 43), pack-reused 0 (from 0)
Receiving objects: 100% (230/230), 1.59 MiB | 5.17 MiB/s, done.
Resolving deltas: 100% (120/120), done.


In [2]:
!pip install pyliferisk

In [3]:
import pandas as pd
import numpy as np

In [4]:
from utils_ucm import clr

In [5]:
from pyliferisk import MortalityTable, lx
from pyliferisk.mortalitytables import PASEM2020_Decesos_M_2ord
mt = MortalityTable(qx=PASEM2020_Decesos_M_2ord)

In [6]:
def incr_capital(capital, t):
  return capital * (1 + 0.015) ** t

In [7]:
escenario = 'Base'
shock_mort = False
shock_c_up = False
shock_c_do = False
shock_c_ma = False
shock_gast = False
shock_cat_ = False

In [11]:
edad = 50
df = pd.DataFrame(pd.date_range(start='2024-12-31',periods=(mt.w -edad),freq='YE'), columns=['Fecha'])
df['edad'] = edad + df.index
df['t'] = df.index
df['lx'] = df['edad'].apply(lambda x: mt.lx[x+1] if x <= mt.w else 0)
df['qx'] = df['lx'].diff(-1).fillna(0)/df['lx'][0]
df['sum_qx'] = df['qx'].cumsum()
df['px'] = 1 - df['sum_qx']
df['capital'] = incr_capital(5000, df.t)
df['pagos'] = df['capital'] * df['qx']
df['caida'] = 0.03
df['polizas'] = (1 - df['caida']).cumprod()
df['clr'] = df['t'].apply(lambda t: clr[t])
df['factor_desc'] = df.apply(lambda x: 1 / (1 + x.clr) ** (x.t), axis=1)
df['polizas_benef'] =  df['px'] * df['polizas']
df['pagos_prob'] = df['pagos'] * df['polizas']
df['gastos'] = df['capital'] * 0.01
df['gastos_prob'] = df['gastos'] * df['polizas_benef']
df['salidas_prob'] = df['pagos_prob'] + df['gastos_prob']
df.salidas_prob @ df.factor_desc

2404.4186468694716

In [9]:
df.to_excel('Flujos2.xlsx', sheet_name=f'{escenario}')
